In [89]:
import tokenizer
import pickle
import pandas as pd
import numpy as np
import glob
import os
import string
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams

In [18]:
path = '/Users/ale/Dropbox (Yadlin Family)/galvanize/capstone/*.txt'

In [19]:
with open("gene_dictionary_lower.pickle", "rb") as dict_gene:
        gene_dict_lower = pickle.load(dict_gene)

In [20]:
gene_keys_lower = set(gene_dict_lower.keys())
gene_values_lower = set(gene_dict_lower.values())

In [21]:
with open("drug_dictionary_lower.pickle", "rb") as dict_drug:
        drug_dict_lower = pickle.load(dict_drug)

In [22]:
drug_keys_lower = set(drug_dict_lower.keys())
drug_values_lower = set(drug_dict_lower.values())

In [23]:
with open("greek_alphabet.pickle", "rb") as dict_greek:
        greek_dict = pickle.load(dict_greek)

In [120]:
output = list(tokenizer.tokenize_many_docs(path))

In [25]:
doc_names, orig_sentences = zip(*output)

In [32]:
corpus = orig_sentences

from sklearn.feature_extraction.text import CountVectorizer

tf = CountVectorizer(tokenizer=tokenizer.tokenize, min_df=0.005)
document_tf_matrix =tf.fit_transform(corpus).todense()

print(sorted(tf.vocabulary_))
print(document_tf_matrix)
#sorted(tf.vocabulary_.items(), key=lambda x: x[1])

['aberr', 'abil', 'activ', 'addit', 'advanc', 'affect', 'age', 'akt1', 'alpha', 'also', 'alter', 'amplif', 'analysi', 'angulo', 'anti', 'antibodi', 'apoptosi', 'approach', 'associ', 'aurora', 'author', 'bind', 'biol', 'biolog', 'black', 'brca1', 'brca2', 'breast', 'cancer', 'carbendazim', 'carcinoma', 'case', 'cell', 'celp', 'certain', 'chang', 'chemotherapi', 'cic', 'clin', 'clinic', 'combin', 'compar', 'complex', 'construct', 'contain', 'contrast', 'control', 'correl', 'cycl', 'cyclin', 'decreas', 'depend', 'detect', 'determin', 'develop', 'differ', 'display', 'dna', 'doi', 'domain', 'doxorubicin', 'drug', 'effect', 'egfr', 'egfr3', 'egfr4', 'egfrviii', 'either', 'elev', 'emt', 'endocrin', 'endometri', 'enhanc', 'epitheli', 'era', 'eralpha', 'erbb2', 'erk', 'esr1', 'estrogen', 'evalu', 'everolimu', 'examin', 'express', 'factor', 'famili', 'figur', 'first', 'follow', 'format', 'fre', 'frequent', 'function', 'gene', 'genet', 'group', 'growth', 'gsk-3', 'gsk-3beta', 'her2', 'high', 'hor

In [143]:
papers_df = pd.DataFrame(document_tf_matrix)
papers_df.columns = tf.get_feature_names()
papers_df.sort_index(axis=1, inplace=True)
papers_df.head()

,aberr,abil,activ,addit,advanc,affect,age,akt1,alpha,also,...,treatment,trial,tumor,tumorigenesi,two,use,variou,well,without,women
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
papers_df.insert(0,'sentences',orig_sentences)
papers_df.insert(0,'files',doc_names)
papers_df.head()

,files,sentences,aberr,abil,activ,addit,advanc,affect,age,akt1,...,treatment,trial,tumor,tumorigenesi,two,use,variou,well,without,women
0,guidelines.txt,Something that moves progressivley First thing...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test.txt,www.impactjournals.com oncotarget Oncotarget...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test.txt,"5, No.",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test.txt,13 Deregulation of the EGFR PI3K PTEN Akt mTO...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,test.txt,ABSTRACT The EGFR PI3K PTEN Akt mTORC1 GSK-3 p...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [145]:
papers_df = papers_df[papers_df[tf.get_feature_names()].any(axis=1)].reset_index().drop('index',axis=1)

In [146]:
papers_df

,files,sentences,aberr,abil,activ,addit,advanc,affect,age,akt1,...,treatment,trial,tumor,tumorigenesi,two,use,variou,well,without,women
0,guidelines.txt,Something that moves progressivley First thing...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test.txt,13 Deregulation of the EGFR PI3K PTEN Akt mTO...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,test.txt,ABSTRACT The EGFR PI3K PTEN Akt mTORC1 GSK-3 p...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,test.txt,The expression of this pathway is frequently a...,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,test.txt,"In some breast cancer cases, mutations at cert...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,test.txt,The expression of this pathway and upstream HE...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,test.txt,The anti-diabetes drug metformin can suppress ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,test.txt,This review will discuss the importance of the...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,test.txt,The targeting of this pathway will be discusse...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
9,test.txt,"The targeting of the hormone receptor, HER2 an...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [100]:
test_df_2.shape

(2893, 265)

In [149]:
def create_data_frame(path,doc_reader = tokenizer.tokenize_many_docs,tokenizer=tokenizer.tokenize, min_df=0.005):
    output = list(doc_reader(path))
    doc_names, orig_sentences = zip(*output)
    corpus = orig_sentences
    tf = CountVectorizer(tokenizer=tokenizer, min_df=min_df)
    document_tf_matrix =tf.fit_transform(corpus).todense()
    papers_df = pd.DataFrame(document_tf_matrix)
    papers_df.columns = tf.get_feature_names()
    papers_df.sort_index(axis=1, inplace=True)
    papers_df.insert(0,'sentences',orig_sentences)
    papers_df.insert(0,'files',doc_names)
    papers_df = papers_df[papers_df[tf.get_feature_names()].any(axis=1)].reset_index().drop('index',axis=1)
    
    return papers_df

In [150]:
create_data_frame(path)

,files,sentences,aberr,abil,activ,addit,advanc,affect,age,akt1,...,treatment,trial,tumor,tumorigenesi,two,use,variou,well,without,women
0,guidelines.txt,Something that moves progressivley First thing...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test.txt,13 Deregulation of the EGFR PI3K PTEN Akt mTO...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,test.txt,ABSTRACT The EGFR PI3K PTEN Akt mTORC1 GSK-3 p...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,test.txt,The expression of this pathway is frequently a...,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,test.txt,"In some breast cancer cases, mutations at cert...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,test.txt,The expression of this pathway and upstream HE...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,test.txt,The anti-diabetes drug metformin can suppress ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,test.txt,This review will discuss the importance of the...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,test.txt,The targeting of this pathway will be discusse...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
9,test.txt,"The targeting of the hormone receptor, HER2 an...",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
